In [5]:
import torch

import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.datasets import fetch_openml
import numpy as np
import sys
sys.path.insert(0, "/home/maria/AMC/includes")
from mlp import *
from torch.utils.data import DataLoader
import math


AttributeError: module 'torch.jit' has no attribute '_script_if_tracing'

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, ix):
        return torch.tensor(self.X[ix]).float(), torch.tensor(self.y[ix]).long()

class MNISTDataModule(pl.LightningDataModule):

    def __init__(self, batch_size: int = 64, Dataset = Dataset):
        super().__init__()
        self.batch_size = batch_size
        self.Dataset = Dataset

    def setup(self, stage=None):
        mnist = fetch_openml('mnist_784', version=1)
        X, y = mnist["data"], mnist["target"]
        X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., y[:60000].astype(np.int), y[60000:].astype(np.int)
        self.train_ds = self.Dataset(X_train, y_train)
        self.val_ds = self.Dataset(X_test, y_test)

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size)


In [ ]:
class AttnDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, patch_size=(7, 7)):
        self.X = X
        self.y = y
        self.patch_size = patch_size

    def __len__(self):
        return len(self.X)

    def __getitem__(self, ix):
        image = torch.tensor(self.X[ix]).float().view(28, 28) # 28 x 28
        h, w = self.patch_size
        patches = image.unfold(0, h, h).unfold(1, w, w) # 4 x 4 x 7 x 7
        patches = patches.contiguous().view(-1, h*w) # 16 x 49
        return patches, torch.tensor(self.y[ix]).long()


In [ ]:
attn_dm = MNISTDataModule(Dataset = AttnDataset)
attn_dm.setup()
imgs, labels = next(iter(attn_dm.train_dataloader()))
print(imgs.shape, labels.shape)
fig = plt.figure(figsize=(5,5))

for i in range(4):
    for j in range(4):
        ax = plt.subplot(4, 4, i*4 + j + 1)
        ax.imshow(imgs[6,i*4 + j].view(7, 7), cmap="gray")
        ax.axis('off')
plt.tight_layout()
plt.show()
